# Lesson 4: Search Index and Boolean Search

In this lesson we'll learn to how build a search index and make basic queries against it.

In [1]:
import re
import sys
import os
import math
import Stemmer

## Preprocessing

This is the same preprocessing that we've been doing up till now.

In [2]:
# The file 'datasets/stopwords_en.txt' contains a list of stopwords - one per line.
with open("../datasets/stopwords_en.txt") as f:
    enStopWords = set(f.read().splitlines())

# Initialze the SnowballStemmer
enStemmer = Stemmer.Stemmer('english')

def preprocess_line_en(line: str) -> list[str]:
    # Convert to lower case
    tokens = line.lower()
    
    # Split into tokens with no punctuation
    tokens = re.split("[^\w]", tokens)
    
    # Remove empty strings and stop words and apply the stemmer
    tokens = [enStemmer.stemWord(x) for x in tokens if x and x not in enStopWords]
    
    # Return the tokens
    return tokens

## Building the Index

Let's build a basic index which for every word in the collection contains the indexes of the documents that contain it.

The collection of documents we will index at first is `datasets/example_search_dataset.xml` one:
```
<document>
<DOC>
	<DOCNO>1</DOCNO>
	<Text>
		He likes to wink, he likes to drink
	</Text>
</DOC>
	
<DOC>
	<DOCNO>2</DOCNO>
	<Text>
		He likes to drink, and drink, and drink
	</Text>
</DOC>

<DOC>
	<DOCNO>3</DOCNO>
	<Text>
		The thing he likes to drink is ink
	</Text>
</DOC>

<DOC>
	<DOCNO>4</DOCNO>
	<Text>
		The ink he likes to drink is pink
	</Text>
</DOC>

<DOC>
	<DOCNO>5</DOCNO>
	<Text>
		He likes to wink, and drink pink ink
	</Text>
</DOC>
</document>
```

## Exercise

Create an inverted index for the above collection of documents in `example_search_dataset.xml`.

You need to save the following information:
- The term (pre-processed) and its document frequency (optional)
- list of documents where this term occured
- for each document, list of positions where the term occured within the document, and the number of term positions (optional)

Print out your index to visualise it.

Example output:
```
drink:5
	1: 4
	2: 2,3,4
	3: 3
	4: 3
	5: 3
ink:3
	3: 4
	4: 1
	5: 5
like:5
	1: 1,3
	2: 1
	3: 2
	4: 2
	5: 1
pink:2
	4: 4
	5: 4
thing:1
	3: 1
wink:2
	1: 2
	5: 2
```

If it's helpful you can use these classes:

In [3]:
# TermPositions represents the positions of a term in a document
# along with the term frequency in that document
class TermPositions():
    def __init__(self, positions):
        self.positions = positions
        self.freq = len(positions)

# IndexTerm is a python dict holding the frequency of each term in
# the index with an extra property containing the document frequency
class IndexTerm(dict):
    def __init__(self, iterable):
        super().__init__(iterable)
        self.docFreq = 0

# Index is a python dict with an extra property containing the document count
class Index(dict):
    def __init__(self, iterable):
        super().__init__(iterable)
        self.docCount = 0

In [ ]:
def create_index(dataset_path):
    # Loop over the dataset file, reading out each document and its Id
    
    # Return the index
    

In [20]:
# Answer
# Create index for the given collection XML file
def create_index(collectionFile):
    # Initialize empty index
    index = Index({})
    docId = 0
    posIdx = 0

    # Parse the XML file line by line and populate the index
    f = open(collectionFile)
    line = f.readline()
    while line:
        # Parse document number
        docNumMatch = re.match('\s*<docno>(\d+)</docno>', line, re.IGNORECASE)
        if docNumMatch:
            docId = int(docNumMatch.group(1))
            
            # Increment the document count
            index.docCount += 1

            # Processing a new document so reset the position index
            posIdx = 0

        # Parse document text or document headline
        elif re.match('\s*<(text|headline)>', line, re.IGNORECASE):
            # Process all the lines in the headline/text
            line = f.readline()
            while not re.match('\s*</(text|headline)>', line, re.IGNORECASE):
                # Preprocess the document text
                terms = preprocess_line_en(line)
            
                # Add each term to the index
                for term in terms:
                    # Increment the term position
                    posIdx += 1
                    if term in index:
                        if docId in index[term]:
                            # If already seen this term in this doc append to the list of positions
                            index[term][docId].positions.append(posIdx)
                            index[term][docId].freq += 1
                        else:
                            # Otherwise create a new entry for this doc under the term
                            index[term][docId] = TermPositions([posIdx])
                    else:
                        # Haven't seen this term before so add to the index
                        index[term] = IndexTerm({docId: TermPositions([posIdx])})

                line = f.readline()
        line = f.readline()

    # Calculate and store the document frequencies
    for term in index:
        index[term].docFreq = len(index[term])
    
    # Close the file
    f.close()

    return index

print("Building index ...")
index = create_index('../datasets/example_search_dataset.xml')
print("Done")

def print_index(index):
# Print the contents of the index
    for term in sorted(index):
        print(f"{term}:{index[term].docFreq}")
        for doc in index[term]:
            print(f"\t{doc}: {','.join([str(x) for x in sorted(index[term][doc].positions)])}")

print_index(index)

Building index ...
Done
drink:5
	1: 4
	2: 2,3,4
	3: 3
	4: 3
	5: 3
ink:3
	3: 4
	4: 1
	5: 5
like:5
	1: 1,3
	2: 1
	3: 2
	4: 2
	5: 1
pink:2
	4: 4
	5: 4
thing:1
	3: 1
wink:2
	1: 2
	5: 2


## Exercise 2 - Querying

Now write a simple function to run a query: given a single word print out the list of documents containing that word.

In [5]:
# Answer
def query_doc_ids(index, term):
    # Query the index for the document ids of documents the given term
    # appears in
    if term and term in index:
        return index[term]
    else:
        return []
    
def single_word_search(word):
    term = preprocess_line_en(word)[0]
    results = query_doc_ids(index, term)
    print(sorted(results))

In [6]:
single_word_search("pink")

[4, 5]


## Exercise 3 - Boolean Search

This is much more advanced :)

1. Write a function to process boolean queries: AND, OR and NOT. e.g. pink AND ink
2. Implement phrase search
3. Implement proximity search

Test it with queries like "wink AND pink", "wink and INK", etc.

In [13]:
def run_boolean_query(query):
    # Run a boolean query on the index and print out the results
    

SyntaxError: incomplete input (613922379.py, line 3)

In [22]:
# Answer
def run_boolean_query(index, query):
    results = []

    # Parse the query
    queryProximity = re.match("#(\d+)\((\w+),\s*(\w+)\)", query)
    queryPhrase = re.match("\"(.+)\"", query)
    queryAndNot = re.match("(.+) AND NOT (.+)", query)
    queryAnd = re.match("(.+) AND (.+)", query)
    queryOrNot = re.match("(.+) OR NOT (.+)", query)
    queryOr = re.match("(.+) OR (.+)", query)
    querySingleTerm = re.match("(\w+)$", query)

    if queryAndNot:
        results1 = run_boolean_query(index, queryAndNot.group(1))
        result2 = run_boolean_query(index, queryAndNot.group(2))
        results = [id for id in results1 if id not in result2]

    elif queryAnd:
        results1 = run_boolean_query(index, queryAnd.group(1))
        result2 = run_boolean_query(index, queryAnd.group(2))
        results = [id for id in results1 if id in result2]

    elif queryOrNot:
        term1 = preprocess_line_en(queryOrNot.group(1))[0]
        term2 = preprocess_line_en(queryOrNot.group(2))[0]
        results1 = query_doc_ids(index, term1)
        # Run through every other term in the index to find doc ids 
        # that do not contain term2. This is super expensive as it 
        # runs through the whole index. 
        # Is there a better way to do this?
        term2DocIds = query_doc_ids(index, term2)
        results2 = [docId for term in index if term != term2 for docId in index[term] if docId not in term2DocIds]
        results = set(results1 + results2)

    elif queryOr:
        results1 = run_boolean_query(index, queryOr.group(1))
        result2 = run_boolean_query(index, queryOr.group(2))
        results = set(results1 + result2)

    elif queryPhrase:
        terms = preprocess_line_en(queryPhrase.group(1))
        results = query_doc_ids(index, terms[0])
        resultPrev = results
        # Loop over each pair of adjacent terms and find doc ids 
        # where the terms are in phrase order and take the intersection
        for i in range(1, len(terms)):
            resultNext = query_doc_ids(index, terms[i])
            resultCurTerms = [id for id in resultPrev if id in resultNext and positionsInPhraseOrder(index[terms[i-1]][id], index[terms[i]][id])]
            results = [docId for docId in results if docId in resultCurTerms]
            resultPrev = resultNext

    elif queryProximity:
        proximity = int(queryProximity.group(1))
        term1 = preprocess_line_en(queryProximity.group(2))[0]
        term2 = preprocess_line_en(queryProximity.group(3))[0]
        results1 = query_doc_ids(index, term1)
        result2 = query_doc_ids(index, term2)
        results = [id for id in results1 if id in result2 and posiitonsInProximity(index[term1][id], index[term2][id], proximity)]

    elif querySingleTerm:
        term = preprocess_line_en(querySingleTerm.group(1))[0]
        results = query_doc_ids(index, term)

    else:
        print(f"Unable to parse query: {query}")
    
    return sorted(results)

def run_query(query):
    results = run_boolean_query(index, query)
    print(results)
    

In [23]:
run_query("pink AND ink")

[4, 5]


## Exercise 4 - Running on TREC dataset

1. Now build and run queries against your inverted index against the TREC dataset: `datasets/trec.5000.xml`
2. Test with more complex boolean queries
3. Verify the results by opening up the dataset file in VSCode and checking the documents contain the search words!

In [16]:
index = create_index('../datasets/trec.5000.xml')

In [12]:
run_query("fish")

[5106, 5128, 5470, 5472, 5474, 5482, 5492, 5507, 5508, 5966, 6050, 6051, 6286, 6439, 6447, 6451, 6456, 6586, 6800, 6903, 6936, 7570, 7595, 7662, 7671, 7677, 7683, 7699, 7708, 7709, 7712, 7713, 7787, 7865, 8057, 8316, 8322, 8324, 8449, 8450, 8451, 8523, 8608, 8741, 8795, 8804, 8805, 8817, 8818, 8822, 8824, 8911, 8912, 8913, 8929, 9035, 9051, 9052, 9053, 9344, 9396, 9433, 9563, 9718, 9793, 10000]
